In [3]:
!pip install einops
!pip install gpytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.0/173.0 kB 26.9 MB/s eta 0:00:00


In [4]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as opt
from torch.distributions.normal import Normal
import gpytorch as gpt
from einops import rearrange
import pandas as pd
import numpy as np
from scipy.stats import lognorm, norm
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, make_scorer, mean_absolute_percentage_error
from statsmodels.tsa.seasonal import seasonal_decompose
from copy import copy, deepcopy
import plotly
from plotly import tools
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from copy import copy, deepcopy
import dask
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
#torch.cuda.set_device(0)
#torch.backends.cudnn.benchmark = True

Mounted at /content/drive


In [5]:
def plot_gp(mu, lb, ub, test_x, test_y, train_x=None, train_y=None, name='', samples={},
            layout='v', xaxis_title='Time', yaxis_title='Sales', fig_size=[1000,500], w=3, f=10):
    fig = make_subplots(rows=1, cols=1, subplot_titles=("Samples"))
    samples = {'sample '+str(i): s for i, s in enumerate(samples)} if not isinstance(samples, dict) else samples
    if train_x is not None:
        fig.add_trace(go.Scatter(x=train_x, y=train_y, mode='lines', name='History', line=dict(width=w), line_color='#1a76ff'))  # plot training data

    fig.add_trace(
        go.Scatter(x=test_x, y=ub, fill=None, mode='lines', line_color='rgba(199, 19, 19, 0.3)',
                   fillcolor='rgba(249, 129, 37, 0.3)', showlegend=True, name='95% uncertainty interval'))
    fig.add_trace(
        go.Scatter(x=test_x, y=lb, fill='tonexty', mode='lines', line_color='rgba(199, 19, 19, 0.3)',
                   fillcolor='rgba(249, 129, 37, 0.3)', showlegend=True, name='95% uncertainty interval'))

    fig.add_trace(go.Scatter(x=test_x, y=mu, line=dict(color='#c71313', width=w), mode='lines', name='Skyolia Forecast'))  # plot the mean
    fig.add_trace(go.Scatter(x=test_x, y=test_y, line=dict(color='#1a76ff', width=w), mode='lines', name='Observed'))
    for k, v in samples.items():
        fig.add_trace(go.Scatter(x=test_x, y=v, name=k, mode='lines',
                                 line=dict(width=w)))  # plot samples
    fig.update_layout(title_text=name, paper_bgcolor='#343434', plot_bgcolor='#343434', xaxis_title=xaxis_title, yaxis_title=yaxis_title,
                          font=dict(family="Montserrat", color="#fff", size=f), title_x=0.5, hovermode="x")
    fig.update_xaxes(showgrid=True, showline=False, gridcolor='#c9c9c9', gridwidth=0.0005)
    fig.update_yaxes(showgrid=True, showline=False, gridcolor='#c9c9c9', gridwidth=0.0005)
    return fig

def confidence_interval(mu, cov):
    std = np.sqrt(np.diag(cov)) #compute std
    uncertainty = 1.96 * std
    return mu, std, mu-uncertainty, mu+uncertainty

def order_quantity(mu, std, cu, co):
    cf = cu/(cu+co)
    return scipy.stats.norm.ppf(cf, loc=mu, scale=std)

def plot_cov(covs, cols, subplot_titles, labels=None):
    fig = make_subplots(rows=int(len(covs)/cols) + 1, cols=cols, subplot_titles=subplot_titles)
    height = (1000/cols)*2
    for i, cov in enumerate(covs):
        row, col = int(i / cols)+1, (i%cols)+1
        fig.add_trace(go.Heatmap(z=cov, x=labels, y=labels, colorscale='Greys'), row=row, col=col)
    fig.update_layout(title_text='Cov matrix', height=height)#, yaxis1=dict(domain=[0, 1]), yaxis1=dict(domain=[0, 1])
    return fig

def plot_ts_decomposition(df, index, obs, model="additive", features=False, period=None, samples=None):
    df.index = df[index]
    decompose = df[[index, obs]]
    decompose.index = df[index]
    decompose = decompose[[obs]]

    decomposition = seasonal_decompose(decompose, model=model, period=period)
    trend, seasonal, residual = decomposition.trend, decomposition.seasonal, decomposition.resid
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=decompose.index, y=decompose.iloc[:,0], mode='lines', name='observed')) #plot the observed
    fig.add_trace(go.Scatter(x=decompose.index, y=trend.tolist(), mode='lines', name='trend')) #plot the trend
    fig.add_trace(go.Scatter(x=decompose.index, y=seasonal.tolist(), mode='lines', name='seasonal')) #plot the seasonal
    fig.add_trace(go.Scatter(x=decompose.index, y=residual.tolist(), mode='lines', name='residual')) #plot the residual
    if features:
        features = [col for col in list(df.columns) if col not in [index, obs]]
        for col in features:
            fig.add_trace(go.Scatter(x=decompose.index, y=df[col].values, name=col, mode='lines'))
    if samples is not None:
        for i, s in enumerate(samples):
            fig.add_trace(go.Scatter(x=decompose.index, y=s, name='sample '+str(i), mode='lines')) #plot samples
    fig.update_layout(title_text='Decomposition')
    return fig, trend.dropna().values, seasonal.dropna().values, residual.dropna().values

def plot_stl_decomposition(df, index, obs, model="additive", period=None, seasonal=7, samples=None):
    df.index = df[index]
    decompose = df[[index, obs]]
    decompose.index = df[index]
    decompose = decompose[[obs]]

    decomposition = STL(decompose, period=period, seasonal=seasonal).fit()
    trend, seasonal, residual = decomposition.trend, decomposition.seasonal, decomposition.resid
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=decompose.index, y=decompose.iloc[:,0], mode='lines', name='observed')) #plot the observed
    fig.add_trace(go.Scatter(x=decompose.index, y=trend.tolist(), mode='lines', name='trend')) #plot the trend
    fig.add_trace(go.Scatter(x=decompose.index, y=seasonal.tolist(), mode='lines', name='seasonal')) #plot the seasonal
    fig.add_trace(go.Scatter(x=decompose.index, y=residual.tolist(), mode='lines', name='residual')) #plot the residual
    if samples is not None:
        for i, s in enumerate(samples):
            fig.add_trace(go.Scatter(x=decompose.index, y=s, name='sample '+str(i), mode='lines')) #plot samples
    fig.update_layout(title_text='Decomposition')
    return fig, trend.dropna().values, seasonal.dropna().values, residual.dropna().values

def date_encoding(df, dt_column, daily=True, weekly=True, yearly=True):
    df[dt_column], periodic_column = df[dt_column].astype('datetime64[ns]'), []
    if daily:
        df['hourofday'] = df[dt_column].dt.hour
        df['sin_hourofday'] = np.sin(2*np.pi*df['hourofday']/np.max(df['hourofday']))
        df['cos_hourofday'] = np.cos(2*np.pi*df['hourofday']/np.max(df['hourofday']))
        df.drop(columns=['hourofday'], inplace=True), periodic_column.extend(['sin_hourofday', 'cos_hourofday'])

    if weekly:
        df['dayofweek'] = df[dt_column].dt.dayofweek
        df['sin_dayofweek'] = np.sin(2*np.pi*df['dayofweek']/np.max(df['dayofweek']))
        df['cos_dayofweek'] = np.cos(2*np.pi*df['dayofweek']/np.max(df['dayofweek']))
        df.drop(columns=['dayofweek'], inplace=True), periodic_column.extend(['sin_dayofweek', 'cos_dayofweek'])
    if yearly:
        df['dayofyear'] = df[dt_column].dt.dayofyear
        df['sin_dayofyear'] = np.sin(2*np.pi*df['dayofyear']/np.max(df['dayofyear']))
        df['cos_dayofyear'] = np.cos(2*np.pi*df['dayofyear']/np.max(df['dayofyear']))
        df.drop(columns=['dayofyear'], inplace=True), periodic_column.extend(['sin_dayofyear', 'cos_dayofyear'])
    return df, periodic_column


def categorical_encoding(train, valid, test, categorical):
    new_cat, LEncoders = [], {}
    for cat in categorical:
        LE = LabelEncoder()
        train[cat] = LE.fit_transform(train[cat]).astype(int)
        valid[cat] = LE.transform(valid[cat]).astype(int)
        test[cat] = LE.transform(test[cat]).astype(int)
        LEncoders[cat] = LE
    return train, valid, test, LEncoders


def numerical_scaling(train, valid, test, numerical):
    MS = MinMaxScaler(feature_range=(-1, 1))
    scaled_train = MS.fit_transform(train[numerical])
    scaled_valid = MS.transform(valid[numerical])
    scaled_test = MS.transform(test[numerical])
    train[numerical] = scaled_train
    valid[numerical] = scaled_valid
    test[numerical] = scaled_test
    return train, valid, test, MS


def dkl_ts_pipeline(train, valid, test, categorical, numerical, periodic, eps=0):
    _train, _valid, _test = train.copy(), valid.copy(), test.copy()
    _train, _valid, _test, LEncoders = categorical_encoding(_train, _valid, _test, categorical)  # CATEGORICAL ENCODING
    if len(numerical) > 0:
        _train, _valid, _test, MS = numerical_scaling(_train, _valid, _test, numerical)  # NUMERCIAL SCALING
    features = categorical + numerical + periodic
    X_train, T_train = _train[features], _train[time_column]
    X_valid, T_valid = _valid[features], _valid[time_column]
    X_test, T_test = _test[features], _test[time_column]
    #Y_train, Y_valid, Y_test, YScaler = output_scaling(train, valid, test, out_column)
    Y_train, Y_valid, Y_test = np.log(_train[out_column].values+eps), np.log(_valid[out_column].values+eps), _test[out_column].values+eps
    return X_train, T_train, Y_train, X_valid, T_valid, Y_valid, X_test, T_test, Y_test


def shift_df(df, shift, dropna=True):
    origin = df.copy()
    for i in range(1, shift+1):
        shifted_df = origin.shift(i)
        shifted_df = shifted_df.rename(columns=dict(zip(shifted_df.columns, [str(c)+'_'+str(i) for c in shifted_df.columns])))
        df = pd.concat([shifted_df, df], axis=1)
    return df.dropna() if dropna else df

def mixture_quantile(pi, mu, std, q):
    pi, mu, std = torch.from_numpy(pi), torch.from_numpy(mu), torch.from_numpy(std)
    n, solutions, evaluations = pi.shape[0], [], []
    for i in range(n):
        mix = Categorical(pi[i])
        comp = LogNormal(mu[i], std[i], validate_args=None)
        pdf = MixtureSameFamily(mix, comp)
        def objf(x):
            x = torch.from_numpy(x)
            return torch.abs(pdf.cdf(x) - q).data.numpy()
        def bounds():
            res = [LogNormal(m, s, validate_args=None).icdf(torch.tensor(q)).data.numpy() for m, s in zip(mu[i], std[0])]
            return np.min(res), np.max(res)

        lb, ub = bounds()
        result = dual_annealing(objf, list(zip([lb], [ub])),maxiter=1000)
        solution = result['x']
        evaluation = objf(solution)
        solutions.append(solution), evaluations.append(evaluation)
    return np.asarray(solutions).ravel(), np.asarray(evaluations)

In [6]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/time series/rossmann sales/sub_rossmann.csv')
df

,Store,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2,Competition
0,45,2015-07-31,6301,442,1,1,0,1,d,a,9710.0,0,1.0
1,45,2015-07-30,6063,445,1,1,0,1,d,a,9710.0,0,1.0
2,45,2015-07-29,5341,387,1,1,0,1,d,a,9710.0,0,1.0
3,45,2015-07-28,5504,365,1,1,0,1,d,a,9710.0,0,1.0
4,45,2015-07-27,7450,500,1,1,0,1,d,a,9710.0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4705,1045,2013-01-05,4854,622,1,0,0,0,a,c,26990.0,0,0.0
4706,1045,2013-01-04,6351,769,1,0,0,1,a,c,26990.0,0,0.0
4707,1045,2013-01-03,7582,923,1,0,0,1,a,c,26990.0,0,0.0
4708,1045,2013-01-02,8282,975,1,0,0,1,a,c,26990.0,0,0.0


In [7]:
out_column, time_column = 'Sales', 'Date'
df[time_column] = pd.to_datetime(df[time_column])

to_remove = ['Customers'] + [c for c in df.columns if df[c].nunique()==1]
features = [c for c in df.columns if (c not in [out_column]) and (c not in to_remove)]
categorical = [c for c in features if (df[c].dtype=='object') and (df[c].nunique() >= 2)] + ['Store']
numerical = [c for c in features if (df[c].dtype=='float') or (df[c].dtype=='int') and (c not in categorical)]

fig = make_subplots(rows=1, cols=2, horizontal_spacing = 0.03, subplot_titles=('Label Distribution', "Features Correlation"))
fig.append_trace(go.Histogram(x=df[out_column]), row=1, col=1)
fig.append_trace(go.Heatmap(z=df[numerical+[out_column]].corr(),x=numerical+[out_column],y=numerical+[out_column]), row=1, col=2)
fig.show()

In [ ]:
sub = df.loc[(df['Store'] == 1045)]
fig, trend, seasonal, residual = plot_ts_decomposition(sub, time_column, out_column, features=True, period=7)
fig.show()
print(np.mean(trend), np.var(trend), np.std(trend))
print(np.mean(seasonal), np.var(seasonal), np.std(seasonal))
print(np.mean(residual), np.var(residual), np.std(residual))

7056.175518925518 2127566.948610514 1458.618164089051
3.6627465284194023 4387582.072148839 2094.6555975025676
-0.20498168967954772 2400761.7794677755 1549.4391822423283


In [8]:
df, periodic = date_encoding(df, time_column, daily=False, weekly=True, yearly=True)
train = df[df[time_column] < '2015-06-01']
valid = df[(df[time_column] >= '2015-06-01') & (df[time_column] < '2015-06-14')]
test = df[df[time_column]>='2015-06-14']

X_train, T_train, Y_train, X_valid, T_valid, Y_valid, X_test, T_test, Y_test = dkl_ts_pipeline(train, valid, test, categorical, numerical, periodic, eps=1e-5)
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((4405, 14), (240, 14), (4405,), (240,))

In [ ]:
X_train

,StateHoliday,StoreType,Assortment,Store,Open,Promo,SchoolHoliday,CompetitionDistance,Promo2,Competition,sin_dayofweek,cos_dayofweek,sin_dayofyear,cos_dayofyear
61,0,2,0,0,-1.0,-1.0,-1.0,-0.511811,-1.0,1.0,-2.449294e-16,1.0,0.516062,-0.856551
62,0,2,0,0,1.0,-1.0,-1.0,-0.511811,-1.0,1.0,-8.660254e-01,0.5,0.530730,-0.847541
63,0,2,0,0,1.0,-1.0,-1.0,-0.511811,-1.0,1.0,-8.660254e-01,-0.5,0.545240,-0.838280
64,0,2,0,0,1.0,-1.0,-1.0,-0.511811,-1.0,1.0,1.224647e-16,-1.0,0.559589,-0.828770
65,0,2,0,0,1.0,-1.0,-1.0,-0.511811,-1.0,1.0,8.660254e-01,-0.5,0.573772,-0.819015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4705,0,0,1,4,1.0,-1.0,-1.0,1.000000,-1.0,-1.0,-8.660254e-01,0.5,0.085965,0.996298
4706,0,0,1,4,1.0,-1.0,1.0,1.000000,-1.0,-1.0,-8.660254e-01,-0.5,0.068802,0.997630
4707,0,0,1,4,1.0,-1.0,1.0,1.000000,-1.0,-1.0,1.224647e-16,-1.0,0.051620,0.998667
4708,0,0,1,4,1.0,-1.0,1.0,1.000000,-1.0,-1.0,8.660254e-01,-0.5,0.034422,0.999407


In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

from torch import autograd

def sinc(x):
    x = x * np.pi
    return torch.where(x != 0., torch.sin(x) / x, 1.)


class RegrDataLoader(Dataset):
    def __init__(self, X, Y, numerical_col, categorical_col):
        self.X1, self.X2, self.Y = X[numerical_col].values.astype(np.float32), X[categorical_col].values, Y.astype(np.float32)

    def __len__(self):
        return len(self.Y)

    def __getitem__(self, idx):
        return self.X1[idx], self.X2[idx], self.Y[idx]

class Embedder(nn.Module):
    def __init__(self, vocab_size, dim):
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, dim)

    def forward(self, x):
        return self.embeddings(x)

class Embedder(nn.Module):
    def __init__(self, embedding_sizes):
        super().__init__()
        self.embeddings = nn.ModuleList([nn.Embedding(categories, size) for categories,size in embedding_sizes])

    def forward(self, x):
        x = [e(x[:,i]) for i,e in enumerate(self.embeddings)]
        return torch.cat(x, 1)

class NNModel(nn.Module):
    def __init__(self, input_shape, units=None, factors=None, activ=True, norm=False, dropout=False, slops=None):
        super().__init__()
        self.input_shape = input_shape
        self.units = units
        self.factors = factors
        self.activ, self.norm = activ, norm
        self.network = nn.ModuleList()
        if self.factors:
            self.units = np.round(self.input_shape * np.asarray(self.factors)).astype(int)
        if self.units is not None:
            self.dropout = np.zeros_like(self.units) if not dropout else dropout
            self.slops = np.full(len(self.units), 1) if slops is None else slops
            for i, j, k in zip(self.units, self.dropout, self.slops):
                if i >= 1:
                    block = self.__build_block__(input_shape, i, p=j, slop=k)
                    self.network.extend(block)
                    input_shape = i
        self.output_shape = input_shape
        self.reset_parameters()

    def __build_block__(self, input_shape, units, p, slop):
        block = []
        block.append(nn.Linear(input_shape, units, bias=not self.norm))
        if self.norm:
            #block.append(nn.BatchNorm1d(units))
            block.append(nn.LayerNorm(units, eps=1e-5))
        if self.activ:
            #block.append(nn.LeakyReLU())
            block.append(nn.ELU(slop))
            #block.append(nn.GELU())
        if p > 0:
            block.append(nn.Dropout(p))
        return block

    def forward(self, x):
        for layer in self.network:
          tmp = layer(x)
          x = tmp
        return x

    def reset_parameters(self):
        for layer in self.network:
            if isinstance(layer, nn.Linear):
                nn.init.xavier_normal_(layer.weight)
                if not self.norm:
                    layer.bias.data.fill_(0.1)


class TabMLP(nn.Module):
    def __init__(self, vocab_size, embed_size, numerical, mlp_units, mlp_dropout=0.00001):
        """
        categories: tuple containing the number of unique values within each category
        """
        super().__init__()
        embedding_sizes = list(zip(vocab_size, embed_size))
        self.cat_embedder = Embedder(embedding_sizes)
        self.norm = nn.BatchNorm1d(numerical, eps=1e-5)
        input_size = numerical + sum(embed_size)
        self.mlp = NNModel(int(input_size), units=mlp_units, factors=None, dropout=[mlp_dropout]*len(mlp_units))
        self.output_shape = mlp_units[-1]

    def forward(self, x_cont, x_cat):
        x = x_cont
        if x_cat.nelement() != 0: #skipped if there's no categorical feature
            x_ = self.cat_embedder(x_cat)
            x = self.norm(x)
            x = torch.cat([x_, x], 1)
        return self.mlp(x)

class DKLModel(gpt.models.ExactGP):
    def __init__(self, feature_extractor, ll_units, covar_module, train_x, train_y, likelihood):
        '''train_x = torch.from_numpy(train_x.values.astype(np.float32)) if train_x is not None else None
        train_y = torch.from_numpy(train_y.astype(np.float32)) if train_y is not None else None'''
        super(DKLModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpt.means.ConstantMean()
        self.covar_module = covar_module
        self.feature_extractor = feature_extractor
        self.last_layer = nn.Sequential(nn.Linear(feature_extractor.output_shape, ll_units))

    def forward(self, x_cont, x_cat):
        x = self.feature_extractor(x_cont, x_cat)
        x = self.last_layer(x)
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpt.distributions.MultivariateNormal(mean_x, covar_x)

class DKL:
    def __init__(self, model, likelihood, numerical_col, categorical_col):
        self.model = model.to(device)
        self.numerical_col, self.categorical_col = numerical_col, categorical_col
        self.losses = {'Epoch': [], 'Train': [], 'Test': [], 'State': [], 'LR': []}
        self.likelihood = likelihood.to(device)
        self.mll = gpt.mlls.ExactMarginalLogLikelihood(self.likelihood, self.model)

    def train_model(self, optim, train_load, grad_clip):
          total_loss = 0
          self.model.train()
          self.likelihood.train()
          for i, (X1, X2, Y) in enumerate(train_load):
              X1, X2, Y = X1.to(device), X2.to(device), Y.to(device)
              self.optim.zero_grad()
              pred = self.model(X1, X2)
              loss = -self.mll(pred, Y)
              loss.backward()
              torch.nn.utils.clip_grad_norm_(self.model.parameters(), grad_clip)
              self.optim.step()
              total_loss += loss.item()
          return total_loss/(i+1)

    def eval_model(self, test_load):
        self.model.eval()
        self.likelihood.eval()
        total_loss = 0
        with torch.no_grad(), gpt.settings.use_toeplitz(False), gpt.settings.fast_pred_var():
            for i, (X1, X2, Y) in enumerate(test_load):
                X1, X2, Y = X1.to(device), X2.to(device), Y.to(device)
                pred = self.model(X1, X2)
                loss = -self.mll(pred, Y)
                total_loss += loss.item()
        return total_loss/(i+1)

    def fit(self, X_train, Y_train, epoch, lr, opt_kwarg, batch_size=None,  grad_clip=100, momentum=0.9, X_test=None, Y_test=None, l2_reg=0, eval=True, verbose=True, save=True):
        batch_size = len(X_train) if batch_size is None else batch_size
        train_load = DataLoader(RegrDataLoader(X_train, Y_train, self.numerical_col, self.categorical_col), batch_size=batch_size, shuffle=False)  # DATALOADER obj
        if X_test is not None:
            test_load = DataLoader(RegrDataLoader(X_test, Y_test, self.numerical_col, self.categorical_col), batch_size=batch_size, shuffle=False)  # DATALOADER obj

        best_loss = 1e100
        self.optim = opt.SGD([{'params': model.feature_extractor.parameters()},
                         {'params': model.covar_module.parameters()},
                         {'params': model.mean_module.parameters()},
                         {'params': model.likelihood.parameters()}], lr=lr, momentum=momentum, nesterov=True)

        '''self.optim = opt.Adam([{'params': model.feature_extractor.parameters()},
                     {'params': model.covar_module.parameters()},
                     {'params': model.mean_module.parameters()},
                     {'params': model.likelihood.parameters()}], lr=lr)'''

        scheduler = None
        scheduler = opt.lr_scheduler.CyclicLR(self.optim, **opt_kwarg)
        #scheduler = opt.lr_scheduler.ReduceLROnPlateau(self.optim, **opt_kwarg)

        for i in range(epoch):
            if verbose:
                print('##### EPOCH ' + str(i) + ' #####')

            try:
                train_loss = self.train_model(self.optim, train_load, grad_clip)
            except gpt.utils.errors.NotPSDError:
                print('EXCEPTION')
                continue

            if verbose:
                print('train loss : ', train_loss)
            self.losses['Epoch'].append(i), self.losses['Train'].append(train_loss)

            if X_test is not None:
                try:
                    valid_loss = self.eval_model(test_load)
                except gpt.utils.errors.NotPSDError:
                    print('EXCEPTION')
                    continue

                if verbose:
                    print('test loss : ', valid_loss)
                self.losses['Test'].append(valid_loss)

                if scheduler is not None:
                    '''scheduler.step(valid_loss)
                    self.losses['LR'].append(self.optim.param_groups[0]['lr'])'''
                    scheduler.step()
                    self.losses['LR'].append(scheduler.get_last_lr()[0])

                if valid_loss < best_loss:
                    self.losses['BState'] = deepcopy(self.model.state_dict())
                    best_loss = valid_loss
                    print('===========SAVE===========')

    def predict(self, X):
        self.model.eval()
        self.likelihood.eval()
        X1 = torch.tensor(X[self.numerical_col].values.astype(np.float32)).to(device)
        X2 = torch.tensor(X[self.categorical_col].values).to(device)
        with torch.no_grad(), gpt.settings.use_toeplitz(False), gpt.settings.fast_pred_var():
            preds = self.likelihood(self.model(X1, X2))
            #-self.mll(preds, torch.from_numpy(y_scaler.transform(test_y[:,np.newaxis].astype(np.float32)).flatten())), preds.covariance_matrix.shape
        mu_test = preds.mean
        std = torch.sqrt(preds.variance)
        cov_test = preds.covariance_matrix
        return mu_test, std, cov_test


def create_kernel(param):
    #kernel = gpt.kernels.ScaleKernel(gpt.kernels.RBFKernel(ard_num_dims=param))
    #kernel = gpt.kernels.ScaleKernel(gpt.kernels.RQKernel(ard_num_dims=param))
    kernel = gpt.kernels.ScaleKernel(gpt.kernels.PiecewisePolynomialKernel(q=0, ard_num_dims=param))
    #kernel = gpt.kernels.ScaleKernel(gpt.kernels.SpectralMixtureKernel(num_mixtures=2, ard_num_dims=param))
    #kernel = gpt.kernels.ScaleKernel(gpt.kernels.PolynomialKernel(power=3))
    return kernel

def gradient_clipper(model: nn.Module, val: float) -> nn.Module:
    def process_grad(grad):
        grad[grad != grad] = 1e-10
        return torch.clamp(grad, -val, val)
    for parameter in model.parameters():
        parameter.register_hook(lambda grad: process_grad(grad))
    return model

In [18]:
ll_units, epoch, lr, batch_size, mlp_d = 16, 50000, 1e-5, None, 1e-6
cyclic_kwarg = {'base_lr': lr, 'max_lr': 1e-2, 'step_size_up':250, 'step_size_down':250}
plateau_kwarg = {'factor':0.5, 'patience':200, 'verbose':True, 'min_lr':1e-7, 'mode':'min'}

embed_size = np.ceil(np.sqrt(list(X_train[categorical].nunique()))).astype(int)
feature_extractor = TabMLP(df[categorical].nunique(), embed_size, len(numerical+periodic), [128, 512, 768, 256], mlp_dropout=mlp_d)
covar_module = create_kernel(ll_units)

likelihood = gpt.likelihoods.GaussianLikelihood()
numerical_torch = torch.from_numpy(X_train[numerical+periodic].values.astype(np.float32))
categorical_torch = torch.from_numpy(X_train[categorical].values.astype(int))
model = gradient_clipper(DKLModel(feature_extractor, ll_units, covar_module, (numerical_torch, categorical_torch), torch.from_numpy(Y_train.astype(np.float32)), likelihood), 10)
dkl = DKL(model, likelihood, numerical+periodic, categorical)
#model.load_state_dict(best_state)
print(model)
print(sum(p.numel() for p in dkl.model.parameters() if p.requires_grad))
dkl.fit(X_train, Y_train, epoch, lr, cyclic_kwarg, batch_size=batch_size, grad_clip=10, momentum=0.9, X_test=X_valid, Y_test=Y_valid, l2_reg=0, eval=True, verbose=True)

DKLModel(
  (likelihood): GaussianLikelihood(
    (noise_covar): HomoskedasticNoise(
      (raw_noise_constraint): GreaterThan(1.000E-04)
    )
  )
  (mean_module): ConstantMean()
  (covar_module): ScaleKernel(
    (base_kernel): PiecewisePolynomialKernel(
      (raw_lengthscale_constraint): Positive()
    )
    (raw_outputscale_constraint): Positive()
  )
  (feature_extractor): TabMLP(
    (cat_embedder): Embedder(
      (embeddings): ModuleList(
        (0): Embedding(4, 2)
        (1): Embedding(3, 2)
        (2): Embedding(2, 2)
        (3): Embedding(5, 3)
      )
    )
    (norm): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (mlp): NNModel(
      (network): ModuleList(
        (0): Linear(in_features=19, out_features=128, bias=True)
        (1): ELU(alpha=1)
        (2): Dropout(p=1e-06, inplace=False)
        (3): Linear(in_features=128, out_features=512, bias=True)
        (4): ELU(alpha=1)
        (5): Dropout(p=1e-06, inplace=False)
    

/usr/local/lib/python3.10/dist-packages/linear_operator/utils/cholesky.py:40: NumericalWarning:

A not p.d., added jitter of 1.0e-06 to the diagonal

/usr/local/lib/python3.10/dist-packages/linear_operator/utils/cholesky.py:40: NumericalWarning:

A not p.d., added jitter of 1.0e-05 to the diagonal

/usr/local/lib/python3.10/dist-packages/linear_operator/utils/cholesky.py:40: NumericalWarning:

A not p.d., added jitter of 1.0e-04 to the diagonal



EXCEPTION
##### EPOCH 1466 #####
train loss :  -0.25647711753845215
test loss :  0.058395855128765106
##### EPOCH 1467 #####
train loss :  -0.2520563304424286
test loss :  0.008254652842879295
##### EPOCH 1468 #####
train loss :  -0.3225701153278351
test loss :  0.008810014463961124
##### EPOCH 1469 #####
train loss :  -0.31648972630500793
test loss :  -0.07003302872180939
##### EPOCH 1470 #####
train loss :  -0.24984747171401978
test loss :  -0.0050080521032214165
##### EPOCH 1471 #####
train loss :  -0.3910536468029022
test loss :  -0.08272293955087662
##### EPOCH 1472 #####
train loss :  -0.44096916913986206
test loss :  0.023932941257953644
##### EPOCH 1473 #####
train loss :  -0.37627318501472473
test loss :  -0.03470071777701378
##### EPOCH 1474 #####
train loss :  -0.33753159642219543
test loss :  0.13212914764881134
##### EPOCH 1475 #####
train loss :  -0.473258912563324
test loss :  -0.05661937966942787
##### EPOCH 1476 #####
train loss :  -0.4568047821521759
test loss :  0.00

/usr/local/lib/python3.10/dist-packages/linear_operator/utils/linear_cg.py:337: NumericalWarning:

CG terminated in 1000 iterations with average residual norm 739.7330322265625 which is larger than the tolerance of 0.01 specified by linear_operator.settings.cg_tolerance. If performance is affected, consider raising the maximum number of CG iterations by running code in a linear_operator.settings.max_cg_iterations(value) context.



test loss :  837100608.0
##### EPOCH 1870 #####
train loss :  -0.5219957232475281
test loss :  0.6955903172492981
##### EPOCH 1871 #####
train loss :  -0.4423312544822693
test loss :  0.2790466248989105
##### EPOCH 1872 #####
train loss :  -0.21670453250408173
test loss :  0.19046659767627716
##### EPOCH 1873 #####
train loss :  -0.30382323265075684
test loss :  0.21547546982765198
##### EPOCH 1874 #####
train loss :  -0.19106151163578033
test loss :  0.26218003034591675
##### EPOCH 1875 #####
train loss :  -0.2799169421195984
test loss :  0.3936448395252228
##### EPOCH 1876 #####
train loss :  -0.4995112717151642
test loss :  0.6135106086730957
##### EPOCH 1877 #####
train loss :  -0.23984363675117493
test loss :  0.37087753415107727
##### EPOCH 1878 #####
train loss :  -0.11173991858959198
test loss :  0.7681018710136414
##### EPOCH 1879 #####
train loss :  -0.18831327557563782
test loss :  0.6167978048324585
##### EPOCH 1880 #####
train loss :  -0.25613394379615784
test loss :  1.83

...

/usr/local/lib/python3.10/dist-packages/linear_operator/utils/linear_cg.py:337: NumericalWarning:

CG terminated in 1000 iterations with average residual norm 6667.01953125 which is larger than the tolerance of 0.01 specified by linear_operator.settings.cg_tolerance. If performance is affected, consider raising the maximum number of CG iterations by running code in a linear_operator.settings.max_cg_iterations(value) context.



test loss :  49043054592.0
##### EPOCH 2324 #####
train loss :  -0.32498103380203247


/usr/local/lib/python3.10/dist-packages/linear_operator/utils/linear_cg.py:337: NumericalWarning:

CG terminated in 1000 iterations with average residual norm 22923.55859375 which is larger than the tolerance of 0.01 specified by linear_operator.settings.cg_tolerance. If performance is affected, consider raising the maximum number of CG iterations by running code in a linear_operator.settings.max_cg_iterations(value) context.



test loss :  478947803136.0
##### EPOCH 2325 #####
train loss :  -0.5975584983825684


KeyboardInterrupt: ignored

In [13]:
best_state = deepcopy(dkl.losses['BState'])
dkl.model.load_state_dict(best_state), np.min(dkl.losses['Test'])
print(np.min(dkl.losses['Test']))
fig = make_subplots(rows=2, cols=1)
s = 0
fig.append_trace(go.Scatter(x=dkl.losses['Epoch'][s:], y=dkl.losses['Train'][s:],mode='lines',name='Train'), row=1, col=1)
fig.append_trace(go.Scatter(x=dkl.losses['Epoch'][s:], y=dkl.losses['Test'][s:],mode='lines',name='Test'), row=1, col=1)
fig.append_trace(go.Scatter(x=dkl.losses['Epoch'][s:], y=dkl.losses['LR'][s:],mode='lines',name='LR'), row=2, col=1)
fig.update_layout(height=1000, width=1500, title_text="Stacked Subplots")
fig.show()

-0.2790168225765228


In [14]:
dkl_mu, dkl_std, dkl_cov = dkl.predict(X_test)
dkl_mu, dkl_std = dkl_mu.cpu().data.numpy(), dkl_std.cpu().data.numpy()
mu_test = np.exp(dkl_mu + np.power(dkl_std, 2) / 2)
std_test = np.sqrt((np.exp(np.power(dkl_std, 2)) - 1) * np.exp(2 * dkl_mu + np.power(dkl_std, 2)))
lb, ub = norm.ppf(0.025, mu_test, std_test), norm.ppf(0.975, mu_test, std_test)
pred = pd.DataFrame({"mu": mu_test, "lb": lb, "ub": ub})
pred = pd.concat((test.reset_index(), pred), 1)
pred

<ipython-input-14-f28ea95c2c18>:7: FutureWarning:

In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.



,index,Store,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,...,CompetitionDistance,Promo2,Competition,sin_dayofweek,cos_dayofweek,sin_dayofyear,cos_dayofyear,mu,lb,ub
0,0,45,2015-07-31,6301,442,1,1,0,1,d,...,9710.0,0,1.0,-8.660254e-01,-0.5,-0.486273,-0.873807,5902.127930,3796.495221,8007.760639
1,1,45,2015-07-30,6063,445,1,1,0,1,d,...,9710.0,0,1.0,1.224647e-16,-1.0,-0.471160,-0.882048,5909.078125,3838.291958,7979.864292
2,2,45,2015-07-29,5341,387,1,1,0,1,d,...,9710.0,0,1.0,8.660254e-01,-0.5,-0.455907,-0.890028,5737.235352,3690.963568,7783.507135
3,3,45,2015-07-28,5504,365,1,1,0,1,d,...,9710.0,0,1.0,8.660254e-01,0.5,-0.440519,-0.897743,5605.695312,3469.381942,7742.008683
4,4,45,2015-07-27,7450,500,1,1,0,1,d,...,9710.0,0,1.0,0.000000e+00,1.0,-0.425000,-0.905193,6808.231445,4000.167933,9616.294958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,3811,1045,2015-06-18,8701,1068,1,1,0,0,a,...,26990.0,0,1.0,1.224647e-16,-1.0,0.230306,-0.973118,8653.404297,4879.858028,12426.950566
236,3812,1045,2015-06-17,8305,973,1,1,0,0,a,...,26990.0,0,1.0,8.660254e-01,-0.5,0.247022,-0.969010,8411.341797,4734.319208,12088.364386
237,3813,1045,2015-06-16,9158,1094,1,1,0,0,a,...,26990.0,0,1.0,8.660254e-01,0.5,0.263665,-0.964614,8693.514648,4961.677906,12425.351391
238,3814,1045,2015-06-15,11109,1224,1,1,0,0,a,...,26990.0,0,1.0,0.000000e+00,1.0,0.280231,-0.959933,9852.357422,5356.738362,14347.976482


In [15]:
def mase(train_y, test_y, pred):
    n = train_y.shape[0]
    d = np.abs(np.diff(train_y)).sum()/(n-1)
    errors = np.abs(test_y - pred)
    return errors.mean()/d

def mape(test_y, pred):
    return np.round(np.mean(np.abs(100*(test_y-pred)/(test_y + 1e-9))), 0)

def rmspe(test_y, pred):
    return (np.sqrt(np.mean(np.square((test_y - pred) / (test_y + 1e-9))))) * 100

def persistence(train_y, test_y):
    predictions, history = [], list(np.copy(train_y))
    for i in test_y:
        predictions.append(history[-1])
        history.append(i)
    return np.asarray(predictions)

def mda(actual, predicted):
    """ Mean Directional Accuracy """
    return np.mean((np.sign(actual[1:] - actual[:-1]) == np.sign(predicted[1:] - predicted[:-1])).astype(int))

def wape(true, pred):
    return np.sum(np.abs(true - pred))/np.sum(true)

naive = persistence(Y_train, Y_test)
errors = {'MAE':[mean_absolute_error(Y_test, pred['mu'].values)],
        'RMSE':[mean_squared_error(Y_test, pred['mu'].values)],
        'MAPE':[mean_absolute_percentage_error(Y_test, pred['mu'].values)],
        'R2': [r2_score(Y_test, pred['mu'].values)],
        'MDA': [mda(Y_test, pred['mu'].values)],
        'WAPE':[wape(Y_test, pred['mu'].values)]}
errors = pd.DataFrame(errors, index =['THIS', 'NAIVE'])
errors

,MAE,RMSE,MAPE,R2,MDA,WAPE
THIS,945.889404,1.874655e+06,0.149321,0.794805,0.794979,0.156748
NAIVE,945.889404,1.874655e+06,0.149321,0.794805,0.794979,0.156748


In [16]:
sub_pred = pred.loc[pred['Store']==45]
sub_ytest, sub_ttest = sub_pred[out_column].values, sub_pred[time_column].values
sub_train = train.loc[train['Store']==45]
sub_ytrain, sub_ttrain = sub_train[out_column].values, sub_train[time_column].values
fig = plot_gp(sub_pred['mu'].values, sub_pred['lb'].values, sub_pred['ub'].values, sub_ttest, sub_ytest, sub_ttrain, sub_ytrain, samples=[], layout='h',
              name='MAE: '+str(mean_absolute_error(sub_ytest, sub_pred['mu'].values))+
              'MAPE: '+str(mean_absolute_percentage_error(sub_ytest, sub_pred['mu'].values))+
              'R2: '+str(r2_score(sub_ytest, sub_pred['mu'].values))+
              'MDA: '+str(mda(sub_ytest, sub_pred['mu'].values))+
              'WAPE: '+str(wape(sub_ytest, sub_pred['mu'].values)))

fig.show()